In [1]:
import sys, re

# https://github.com/fzinfz/meter-dlt645
sys.path.append("/data/powermeter_dlt645/meter-dlt645")

from test_dlt645 import *
import test_dlt645

In [2]:
verbose=0

serial_port = '/dev/ttyUSB0'
timeout_count = 10
wait_for_read = 0.5

chn=dlt645.Channel(port_id = serial_port, tmo_cnt = timeout_count, wait_for_read = wait_for_read)
chn.open()

True

In [3]:
read_meter_address(chn, verbose)


--- Read meter address with broadcase mode ---
Meter address: 000102866265


1

In [4]:
addr = chn.rx_addr
print(addr)

addr_human = ''.join([ '{0:0>2x}'.format(s) for s in addr ])
print(addr_human)

[ int(s,16) for s in re.findall('..',addr_human) ]

[0, 1, 2, 134, 98, 101]
000102866265


[0, 1, 2, 134, 98, 101]

In [5]:
read_date(chn, addr, verbose)
read_time(chn, addr, verbose)    

read_voltage(chn, addr, verbose)
read_current(chn, addr, verbose)
read_power(chn, addr, verbose)    
# read_temperature(chn, addr, verbose)
read_battery_voltage(chn, addr, verbose)

read_last_outage_timestamp(chn, addr, 1, verbose)
# read_preset_billing_time(chn, addr, verbose)

# read_power_energy(chn, addr, verbose)

for m in range(0,2):
    for n in [0,2,4]:
        read_energy(chn, addr, m, n, verbose)


--- Read date ---
Date: 20-10-30 Fri

--- Read time ---
Time: 16:45:29

--- Read voltage ---
Voltage: 229.4 V

--- Read current ---
Current: 0.482 A

--- Read power ---
Power: 0.0314 kW

--- Read battery voltage ---
Battery voltage: 3.65 V

--- Read last outage timestamp of N = 1 ---
Last outage timestamp: 20-10-28 15:37:28 ----> 20-10-28 15:38:56 

--- Read energy (Current Month) ---
Total: 19910.76 kWh

--- Read energy (Current Month) ---
Peak: 12090.54 kWh

--- Read energy (Current Month) ---
Trough: 7820.22 kWh

--- Read energy (Last 1 Month) ---
Total: 19483.45 kWh

--- Read energy (Last 1 Month) ---
Peak: 11785.79 kWh

--- Read energy (Last 1 Month) ---
Trough: 7697.65 kWh


In [6]:
chn.close()